# Image Scaling 
<li>Mean Subtraction</li>
<li>Normalization</li>

Step 1: Import libraries and Fashion mnist Datasets

In [115]:
from keras.datasets import fashion_mnist
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
# load dataset  datasets..load_data()
(trainX, trainY), (testX, testY) = fashion_mnist.load_data()

Step 2: Reshaping datasets

In [116]:
width, height, channels = trainX.shape[1], trainX.shape[2], 1
trainX = trainX.reshape((trainX.shape[0], width, height, channels))
testX = testX.reshape((testX.shape[0], width, height, channels))

Step 3: One hot encode and Calculating Mean using ImageDataGenerator Function

In [117]:
# reshape dataset to have a single channel
width, height, channels = trainX.shape[1], trainX.shape[2], 1
trainX = trainX.reshape((trainX.shape[0], width, height, channels))
testX = testX.reshape((testX.shape[0], width, height, channels))
# one hot encode target values
trainY = to_categorical(trainY)
testY = to_categorical(testY)


Step 4: Creating Convolution Neural Network Model

In [59]:
def MeanSubtraction():
    # create generator to center images
    datagen = ImageDataGenerator(featurewise_center=True)
    # calculate mean on training dataset
    datagen.fit(trainX)
    # prepare an iterators to scale images
    train_iterator = datagen.flow(trainX, trainY, batch_size=64)
    test_iterator = datagen.flow(testX, testY, batch_size=64)
    print('Batches train=%d, test=%d' % (len(train_iterator), len(test_iterator)))
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(width, height, channels)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(10, activation='softmax'))
    # compile model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    # fit model with generator
    model.fit_generator(train_iterator, steps_per_epoch=len(train_iterator), epochs=5)
    # evaluate model
    _, acc = model.evaluate_generator(test_iterator, steps=len(test_iterator), verbose=0)
    print('Test Accuracy: %.3f' % (acc * 100))

In [60]:
MeanSubtraction()

Batches train=938, test=157
Epoch 1/5
938/938 [==============================] - 17s 18ms/step - loss: 1.5976 - accuracy: 0.7391
Epoch 2/5
938/938 [==============================] - 17s 18ms/step - loss: 0.3555 - accuracy: 0.8707
Epoch 3/5
938/938 [==============================] - 17s 18ms/step - loss: 0.3057 - accuracy: 0.8868
Epoch 4/5
938/938 [==============================] - 17s 18ms/step - loss: 0.2783 - accuracy: 0.8965
Epoch 5/5
938/938 [==============================] - 17s 18ms/step - loss: 0.2593 - accuracy: 0.9043
Test Accuracy: 88.030


Step 5: Normailization using informations in step 1 and 2

In [63]:
def Normalization():
    # confirm scale of pixels
    print('Train min=%.3f, max=%.3f' % (trainX.min(), trainX.max()))
    print('Test min=%.3f, max=%.3f' % (testX.min(), testX.max()))
    # create generator (1.0/255.0 = 0.003921568627451)
    datagen = ImageDataGenerator(rescale=1.0/255.0)
    # prepare an iterators to scale images
    train_iterator = datagen.flow(trainX, trainY, batch_size=64)
    test_iterator = datagen.flow(testX, testY, batch_size=64)
    print('Batches train=%d, test=%d' % (len(train_iterator), len(test_iterator)))
    # confirm the scaling works
    batchX, batchy = train_iterator.next()
    print('Batch shape=%s, min=%.3f, max=%.3f' % (batchX.shape, batchX.min(), batchX.max()))
    # define model
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(width, height, channels)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(10, activation='softmax'))
    # compile model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    # fit model with generator
    model.fit_generator(train_iterator, steps_per_epoch=len(train_iterator), epochs=5)
    # evaluate model
    _, acc = model.evaluate_generator(test_iterator, steps=len(test_iterator), verbose=0)
    print('Test Accuracy: %.3f' % (acc * 100))
    

In [64]:
Normalization()

Train min=0.000, max=255.000
Test min=0.000, max=255.000
Batches train=938, test=157
Batch shape=(64, 28, 28, 1), min=0.000, max=1.000
Epoch 1/5
938/938 [==============================] - 17s 18ms/step - loss: 0.7182 - accuracy: 0.7399
Epoch 2/5
938/938 [==============================] - 17s 18ms/step - loss: 0.3373 - accuracy: 0.8786
Epoch 3/5
938/938 [==============================] - 17s 18ms/step - loss: 0.2862 - accuracy: 0.8955
Epoch 4/5
938/938 [==============================] - 16s 18ms/step - loss: 0.2577 - accuracy: 0.9046
Epoch 5/5
938/938 [==============================] - 16s 17ms/step - loss: 0.2309 - accuracy: 0.9153
Test Accuracy: 90.500


# weights Initialization
<li>Xavier Initialization</li>
<li>He Initialization</li>

Case 1: Xavier Initialization using Normalized data

In [65]:

def XavierInitialization():
    # confirm scale of pixels
    print('Train min=%.3f, max=%.3f' % (trainX.min(), trainX.max()))
    print('Test min=%.3f, max=%.3f' % (testX.min(), testX.max()))
    # create generator (1.0/255.0 = 0.003921568627451)
    datagen = ImageDataGenerator(rescale=1.0/255.0)
    # prepare an iterators to scale images
    train_iterator = datagen.flow(trainX, trainY, batch_size=64)
    test_iterator = datagen.flow(testX, testY, batch_size=64)
    print('Batches train=%d, test=%d' % (len(train_iterator), len(test_iterator)))
    # confirm the scaling works
    batchX, batchy = train_iterator.next()
    print('Batch shape=%s, min=%.3f, max=%.3f' % (batchX.shape, batchX.min(), batchX.max()))
    # define model
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(width, height, channels),
                    kernel_initializer = tf.keras.initializers.GlorotNormal()))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu',kernel_initializer = tf.keras.initializers.GlorotNormal()))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(64, activation='relu',kernel_initializer = tf.keras.initializers.GlorotNormal()))
    model.add(Dense(10, activation='softmax'))
    # compile model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    # fit model with generator
    model.fit_generator(train_iterator, steps_per_epoch=len(train_iterator), epochs=5)
    # evaluate model
    _, acc = model.evaluate_generator(test_iterator, steps=len(test_iterator), verbose=0)
    print('Test Accuracy: %.3f' % (acc * 100))

In [66]:
XavierInitialization()

Train min=0.000, max=255.000
Test min=0.000, max=255.000
Batches train=938, test=157
Batch shape=(64, 28, 28, 1), min=0.000, max=1.000
Epoch 1/5
938/938 [==============================] - 17s 18ms/step - loss: 0.7049 - accuracy: 0.7541
Epoch 2/5
938/938 [==============================] - 17s 18ms/step - loss: 0.3395 - accuracy: 0.8765
Epoch 3/5
938/938 [==============================] - 17s 18ms/step - loss: 0.2898 - accuracy: 0.8941
Epoch 4/5
938/938 [==============================] - 17s 18ms/step - loss: 0.2530 - accuracy: 0.9081
Epoch 5/5
938/938 [==============================] - 17s 18ms/step - loss: 0.2299 - accuracy: 0.9154
Test Accuracy: 90.080


Case 2: He  Initialization using Normalized data

In [74]:
def HeInitialization():
    # confirm scale of pixels
    print('Train min=%.3f, max=%.3f' % (trainX.min(), trainX.max()))
    print('Test min=%.3f, max=%.3f' % (testX.min(), testX.max()))
    # create generator (1.0/255.0 = 0.003921568627451)
    datagen = ImageDataGenerator(rescale=1.0/255.0)
    # prepare an iterators to scale images
    train_iterator = datagen.flow(trainX, trainY, batch_size=64)
    test_iterator = datagen.flow(testX, testY, batch_size=64)
    print('Batches train=%d, test=%d' % (len(train_iterator), len(test_iterator)))
    # confirm the scaling works
    batchX, batchy = train_iterator.next()
    print('Batch shape=%s, min=%.3f, max=%.3f' % (batchX.shape, batchX.min(), batchX.max()))
    # define model
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(width, height, channels),
                    kernel_initializer = tf.keras.initializers.HeNormal()))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu',kernel_initializer = tf.keras.initializers.HeNormal()))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(64, activation='relu',kernel_initializer = tf.keras.initializers.HeNormal()))
    model.add(Dense(10, activation='softmax'))
    # compile model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    # fit model with generator
    model.fit_generator(train_iterator, steps_per_epoch=len(train_iterator), epochs=5)
    # evaluate model
    _, acc = model.evaluate_generator(test_iterator, steps=len(test_iterator), verbose=0)
    print('Test Accuracy: %.3f' % (acc * 100))

In [75]:
HeInitialization()

Train min=0.000, max=255.000
Test min=0.000, max=255.000
Batches train=938, test=157
Batch shape=(64, 28, 28, 1), min=0.000, max=1.000
Epoch 1/5
938/938 [==============================] - 15s 16ms/step - loss: 0.6084 - accuracy: 0.7819
Epoch 2/5
938/938 [==============================] - 15s 16ms/step - loss: 0.3129 - accuracy: 0.8861
Epoch 3/5
938/938 [==============================] - 15s 16ms/step - loss: 0.2649 - accuracy: 0.9042
Epoch 4/5
938/938 [==============================] - 15s 16ms/step - loss: 0.2333 - accuracy: 0.9139
Epoch 5/5
938/938 [==============================] - 15s 16ms/step - loss: 0.2082 - accuracy: 0.9231
Test Accuracy: 90.250


# Layer Configuration
<li>1 input layer: 2 hidden Layer : 1 output Layer</li>
<li>1 input layer: 3 hidden Layer : 1 output Layer</li>

Case 1: 1 input layer: 2 hidden Layer : 1 output Layer

In [85]:
def Configuration1():
    # confirm scale of pixels
    print('Train min=%.3f, max=%.3f' % (trainX.min(), trainX.max()))
    print('Test min=%.3f, max=%.3f' % (testX.min(), testX.max()))
    # create generator (1.0/255.0 = 0.003921568627451)
    datagen = ImageDataGenerator(rescale=1.0/255.0)
    # prepare an iterators to scale images
    train_iterator = datagen.flow(trainX, trainY, batch_size=64)
    test_iterator = datagen.flow(testX, testY, batch_size=64)
    print('Batches train=%d, test=%d' % (len(train_iterator), len(test_iterator)))
    # confirm the scaling works
    batchX, batchy = train_iterator.next()
    print('Batch shape=%s, min=%.3f, max=%.3f' % (batchX.shape, batchX.min(), batchX.max()))
    # define model
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(width, height, channels),
                    kernel_initializer = tf.keras.initializers.HeNormal()))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu',kernel_initializer = tf.keras.initializers.HeNormal()))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(64, activation='relu',kernel_initializer = tf.keras.initializers.HeNormal()))
    model.add(Dense(10, activation='softmax'))
    # compile model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    # fit model with generator
    model.fit_generator(train_iterator, steps_per_epoch=len(train_iterator), epochs=5)
    # evaluate model
    _, acc = model.evaluate_generator(test_iterator, steps=len(test_iterator), verbose=0)
    print('Test Accuracy: %.3f' % (acc * 100))

In [86]:
Configuration1()

Train min=0.000, max=255.000
Test min=0.000, max=255.000
Batches train=938, test=157
Batch shape=(64, 28, 28, 1), min=0.000, max=1.000
Epoch 1/5
938/938 [==============================] - 16s 17ms/step - loss: 0.6158 - accuracy: 0.7735
Epoch 2/5
938/938 [==============================] - 16s 17ms/step - loss: 0.3220 - accuracy: 0.8826
Epoch 3/5
938/938 [==============================] - 16s 17ms/step - loss: 0.2717 - accuracy: 0.8996
Epoch 4/5
938/938 [==============================] - 16s 17ms/step - loss: 0.2344 - accuracy: 0.9120
Epoch 5/5
938/938 [==============================] - 16s 17ms/step - loss: 0.2114 - accuracy: 0.9239
Test Accuracy: 90.450


Case 2: 1 input layer: 3 hidden Layer : 1 output Layer

In [89]:
def Configuration2():
    # confirm scale of pixels
    print('Train min=%.3f, max=%.3f' % (trainX.min(), trainX.max()))
    print('Test min=%.3f, max=%.3f' % (testX.min(), testX.max()))
    # create generator (1.0/255.0 = 0.003921568627451)
    datagen = ImageDataGenerator(rescale=1.0/255.0)
    # prepare an iterators to scale images
    train_iterator = datagen.flow(trainX, trainY, batch_size=64)
    test_iterator = datagen.flow(testX, testY, batch_size=64)
    print('Batches train=%d, test=%d' % (len(train_iterator), len(test_iterator)))
    # confirm the scaling works
    batchX, batchy = train_iterator.next()
    print('Batch shape=%s, min=%.3f, max=%.3f' % (batchX.shape, batchX.min(), batchX.max()))
    # define model
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(width, height, channels),
                    kernel_initializer = tf.keras.initializers.HeNormal()))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu',kernel_initializer = tf.keras.initializers.HeNormal()))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu',kernel_initializer = tf.keras.initializers.HeNormal()))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(64, activation='relu',kernel_initializer = tf.keras.initializers.HeNormal()))
    model.add(Dense(10, activation='softmax'))
    # compile model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    # fit model with generator
    model.fit_generator(train_iterator, steps_per_epoch=len(train_iterator), epochs=5)
    # evaluate model
    _, acc = model.evaluate_generator(test_iterator, steps=len(test_iterator), verbose=0)
    print('Test Accuracy: %.3f' % (acc * 100))

In [90]:
Configuration2()

Train min=0.000, max=255.000
Test min=0.000, max=255.000
Batches train=938, test=157
Batch shape=(64, 28, 28, 1), min=0.000, max=1.000
Epoch 1/5
938/938 [==============================] - 18s 18ms/step - loss: 0.7685 - accuracy: 0.7245
Epoch 2/5
938/938 [==============================] - 17s 18ms/step - loss: 0.3962 - accuracy: 0.8555
Epoch 3/5
938/938 [==============================] - 17s 18ms/step - loss: 0.3319 - accuracy: 0.8796
Epoch 4/5
938/938 [==============================] - 17s 18ms/step - loss: 0.2930 - accuracy: 0.8914
Epoch 5/5
938/938 [==============================] - 17s 18ms/step - loss: 0.2695 - accuracy: 0.9001
Test Accuracy: 88.470


# RMSprop Optimization



In [92]:
def RMSpropOptimization():
    # confirm scale of pixels
    print('Train min=%.3f, max=%.3f' % (trainX.min(), trainX.max()))
    print('Test min=%.3f, max=%.3f' % (testX.min(), testX.max()))
    # create generator (1.0/255.0 = 0.003921568627451)
    datagen = ImageDataGenerator(rescale=1.0/255.0)
    # prepare an iterators to scale images
    train_iterator = datagen.flow(trainX, trainY, batch_size=64)
    test_iterator = datagen.flow(testX, testY, batch_size=64)
    print('Batches train=%d, test=%d' % (len(train_iterator), len(test_iterator)))
    # confirm the scaling works
    batchX, batchy = train_iterator.next()
    print('Batch shape=%s, min=%.3f, max=%.3f' % (batchX.shape, batchX.min(), batchX.max()))
    # define model
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(width, height, channels),
                    kernel_initializer = tf.keras.initializers.HeNormal()))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu',kernel_initializer = tf.keras.initializers.HeNormal()))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(64, activation='relu',kernel_initializer = tf.keras.initializers.HeNormal()))
    model.add(Dense(10, activation='softmax'))
    # compile model
    model.compile(optimizer='RMSprop', loss='categorical_crossentropy', metrics=['accuracy'])
    # fit model with generator
    model.fit_generator(train_iterator, steps_per_epoch=len(train_iterator), epochs=5)
    # evaluate model
    _, acc = model.evaluate_generator(test_iterator, steps=len(test_iterator), verbose=0)
    print('Test Accuracy: %.3f' % (acc * 100))

In [93]:
RMSpropOptimization()

Train min=0.000, max=255.000
Test min=0.000, max=255.000
Batches train=938, test=157
Batch shape=(64, 28, 28, 1), min=0.000, max=1.000
Epoch 1/5
938/938 [==============================] - 16s 17ms/step - loss: 0.6533 - accuracy: 0.7685
Epoch 2/5
938/938 [==============================] - 16s 17ms/step - loss: 0.3110 - accuracy: 0.8862
Epoch 3/5
938/938 [==============================] - 16s 17ms/step - loss: 0.2621 - accuracy: 0.9032
Epoch 4/5
938/938 [==============================] - 16s 17ms/step - loss: 0.2271 - accuracy: 0.9178
Epoch 5/5
938/938 [==============================] - 16s 17ms/step - loss: 0.2044 - accuracy: 0.9240
Test Accuracy: 90.490


# Adadelta Optimization

In [94]:
def Adadeltatimization():
    # confirm scale of pixels
    print('Train min=%.3f, max=%.3f' % (trainX.min(), trainX.max()))
    print('Test min=%.3f, max=%.3f' % (testX.min(), testX.max()))
    # create generator (1.0/255.0 = 0.003921568627451)
    datagen = ImageDataGenerator(rescale=1.0/255.0)
    # prepare an iterators to scale images
    train_iterator = datagen.flow(trainX, trainY, batch_size=64)
    test_iterator = datagen.flow(testX, testY, batch_size=64)
    print('Batches train=%d, test=%d' % (len(train_iterator), len(test_iterator)))
    # confirm the scaling works
    batchX, batchy = train_iterator.next()
    print('Batch shape=%s, min=%.3f, max=%.3f' % (batchX.shape, batchX.min(), batchX.max()))
    # define model
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(width, height, channels),
                    kernel_initializer = tf.keras.initializers.HeNormal()))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu',kernel_initializer = tf.keras.initializers.HeNormal()))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(64, activation='relu',kernel_initializer = tf.keras.initializers.HeNormal()))
    model.add(Dense(10, activation='softmax'))
    # compile model
    model.compile(optimizer='Adadelta', loss='categorical_crossentropy', metrics=['accuracy'])
    # fit model with generator
    model.fit_generator(train_iterator, steps_per_epoch=len(train_iterator), epochs=5)
    # evaluate model
    _, acc = model.evaluate_generator(test_iterator, steps=len(test_iterator), verbose=0)
    print('Test Accuracy: %.3f' % (acc * 100))

In [95]:
Adadeltatimization()

Train min=0.000, max=255.000
Test min=0.000, max=255.000
Batches train=938, test=157
Batch shape=(64, 28, 28, 1), min=0.000, max=1.000
Epoch 1/5
938/938 [==============================] - 16s 17ms/step - loss: 2.5760 - accuracy: 0.18050s - loss: 2.5784 - accuracy: 0.
Epoch 2/5
938/938 [==============================] - 16s 17ms/step - loss: 1.8561 - accuracy: 0.3937
Epoch 3/5
938/938 [==============================] - 16s 17ms/step - loss: 1.5922 - accuracy: 0.5324
Epoch 4/5
938/938 [==============================] - 16s 17ms/step - loss: 1.3887 - accuracy: 0.6075
Epoch 5/5
938/938 [==============================] - 16s 17ms/step - loss: 1.2341 - accuracy: 0.6546
Test Accuracy: 66.860


# SeLU Activation 

In [96]:
def SeLUActivation():
    # confirm scale of pixels
    print('Train min=%.3f, max=%.3f' % (trainX.min(), trainX.max()))
    print('Test min=%.3f, max=%.3f' % (testX.min(), testX.max()))
    # create generator (1.0/255.0 = 0.003921568627451)
    datagen = ImageDataGenerator(rescale=1.0/255.0)
    # prepare an iterators to scale images
    train_iterator = datagen.flow(trainX, trainY, batch_size=64)
    test_iterator = datagen.flow(testX, testY, batch_size=64)
    print('Batches train=%d, test=%d' % (len(train_iterator), len(test_iterator)))
    # confirm the scaling works
    batchX, batchy = train_iterator.next()
    print('Batch shape=%s, min=%.3f, max=%.3f' % (batchX.shape, batchX.min(), batchX.max()))
    # define model
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='selu', input_shape=(width, height, channels),
                    kernel_initializer = tf.keras.initializers.HeNormal()))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='selu',kernel_initializer = tf.keras.initializers.HeNormal()))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(64, activation='selu',kernel_initializer = tf.keras.initializers.HeNormal()))
    model.add(Dense(10, activation='softmax'))
    # compile model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    # fit model with generator
    model.fit_generator(train_iterator, steps_per_epoch=len(train_iterator), epochs=5)
    # evaluate model
    _, acc = model.evaluate_generator(test_iterator, steps=len(test_iterator), verbose=0)
    print('Test Accuracy: %.3f' % (acc * 100))

In [97]:
SeLUActivation()

Train min=0.000, max=255.000
Test min=0.000, max=255.000
Batches train=938, test=157
Batch shape=(64, 28, 28, 1), min=0.000, max=1.000
Epoch 1/5
938/938 [==============================] - 17s 18ms/step - loss: 0.5710 - accuracy: 0.7995
Epoch 2/5
938/938 [==============================] - 17s 18ms/step - loss: 0.2980 - accuracy: 0.8904
Epoch 3/5
938/938 [==============================] - 17s 18ms/step - loss: 0.2494 - accuracy: 0.9107
Epoch 4/5
938/938 [==============================] - 17s 18ms/step - loss: 0.2116 - accuracy: 0.9224
Epoch 5/5
938/938 [==============================] - 17s 18ms/step - loss: 0.1826 - accuracy: 0.9323
Test Accuracy: 89.730


# PReLU Activation 

In [102]:
from keras.layers.advanced_activations import PReLU
def PReLUActivation():
    # confirm scale of pixels
    print('Train min=%.3f, max=%.3f' % (trainX.min(), trainX.max()))
    print('Test min=%.3f, max=%.3f' % (testX.min(), testX.max()))
    # create generator (1.0/255.0 = 0.003921568627451)
    datagen = ImageDataGenerator(rescale=1.0/255.0)
    # prepare an iterators to scale images
    train_iterator = datagen.flow(trainX, trainY, batch_size=64)
    test_iterator = datagen.flow(testX, testY, batch_size=64)
    print('Batches train=%d, test=%d' % (len(train_iterator), len(test_iterator)))
    # confirm the scaling works
    batchX, batchy = train_iterator.next()
    print('Batch shape=%s, min=%.3f, max=%.3f' % (batchX.shape, batchX.min(), batchX.max()))
    # define model
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation=PReLU(), input_shape=(width, height, channels),
                    kernel_initializer = tf.keras.initializers.HeNormal()))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation=PReLU(),kernel_initializer = tf.keras.initializers.HeNormal()))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(64, activation=PReLU(),kernel_initializer = tf.keras.initializers.HeNormal()))
    model.add(Dense(10, activation='softmax'))
    # compile model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    # fit model with generator
    model.fit_generator(train_iterator, steps_per_epoch=len(train_iterator), epochs=5)
    # evaluate model
    _, acc = model.evaluate_generator(test_iterator, steps=len(test_iterator), verbose=0)
    print('Test Accuracy: %.3f' % (acc * 100))

In [103]:
PReLUActivation()

Train min=0.000, max=255.000
Test min=0.000, max=255.000
Batches train=938, test=157
Batch shape=(64, 28, 28, 1), min=0.000, max=1.000
Epoch 1/5
938/938 [==============================] - 32s 34ms/step - loss: 0.6309 - accuracy: 0.7777
Epoch 2/5
938/938 [==============================] - 32s 34ms/step - loss: 0.3266 - accuracy: 0.8822
Epoch 3/5
938/938 [==============================] - 32s 34ms/step - loss: 0.2757 - accuracy: 0.8982
Epoch 4/5
938/938 [==============================] - 31s 34ms/step - loss: 0.2429 - accuracy: 0.9121
Epoch 5/5
938/938 [==============================] - 32s 34ms/step - loss: 0.2132 - accuracy: 0.9229
Test Accuracy: 90.200


# L1 Regularization

In [118]:

def L1Regularization():
    # confirm scale of pixels
    print('Train min=%.3f, max=%.3f' % (trainX.min(), trainX.max()))
    print('Test min=%.3f, max=%.3f' % (testX.min(), testX.max()))
    # create generator (1.0/255.0 = 0.003921568627451)
    datagen = ImageDataGenerator(rescale=1.0/255.0)
    # prepare an iterators to scale images
    train_iterator = datagen.flow(trainX, trainY, batch_size=64)
    test_iterator = datagen.flow(testX, testY, batch_size=64)
    print('Batches train=%d, test=%d' % (len(train_iterator), len(test_iterator)))
    # confirm the scaling works
    batchX, batchy = train_iterator.next()
    print('Batch shape=%s, min=%.3f, max=%.3f' % (batchX.shape, batchX.min(), batchX.max()))
    # define model
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='selu', input_shape=(width, height, channels),
                    kernel_initializer = tf.keras.initializers.HeNormal(),
                     kernel_regularizer=tf.keras.regularizers.l1(0.01)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='selu',kernel_initializer = tf.keras.initializers.HeNormal(),
                     kernel_regularizer=tf.keras.regularizers.l1(0.01)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(64, activation='selu',kernel_initializer = tf.keras.initializers.HeNormal(),
                    kernel_regularizer=tf.keras.regularizers.l1(0.01)))
    model.add(Dense(10, activation='softmax'))
    # compile model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    # fit model with generator
    model.fit_generator(train_iterator, steps_per_epoch=len(train_iterator), epochs=5)
    # evaluate model
    _, acc = model.evaluate_generator(test_iterator, steps=len(test_iterator), verbose=0)
    print('Test Accuracy: %.3f' % (acc * 100))

In [119]:
L1Regularization()

Train min=0.000, max=255.000
Test min=0.000, max=255.000
Batches train=938, test=157
Batch shape=(64, 28, 28, 1), min=0.000, max=1.000
Epoch 1/5
938/938 [==============================] - 18s 19ms/step - loss: 12.0934 - accuracy: 0.7181
Epoch 2/5
938/938 [==============================] - 17s 19ms/step - loss: 1.2401 - accuracy: 0.7817
Epoch 3/5
938/938 [==============================] - 17s 19ms/step - loss: 1.0683 - accuracy: 0.7935
Epoch 4/5
938/938 [==============================] - 18s 19ms/step - loss: 1.0352 - accuracy: 0.8003
Epoch 5/5
938/938 [==============================] - 17s 19ms/step - loss: 1.0038 - accuracy: 0.8038
Test Accuracy: 80.250


# L2 Regularization

In [123]:
def L2Regularization():
    # confirm scale of pixels
    print('Train min=%.3f, max=%.3f' % (trainX.min(), trainX.max()))
    print('Test min=%.3f, max=%.3f' % (testX.min(), testX.max()))
    # create generator (1.0/255.0 = 0.003921568627451)
    datagen = ImageDataGenerator(rescale=1.0/255.0)
    # prepare an iterators to scale images
    train_iterator = datagen.flow(trainX, trainY, batch_size=64)
    test_iterator = datagen.flow(testX, testY, batch_size=64)
    print('Batches train=%d, test=%d' % (len(train_iterator), len(test_iterator)))
    # confirm the scaling works
    batchX, batchy = train_iterator.next()
    print('Batch shape=%s, min=%.3f, max=%.3f' % (batchX.shape, batchX.min(), batchX.max()))
    # define model
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(width, height, channels),
                    kernel_initializer = tf.keras.initializers.HeNormal(),
                     kernel_regularizer=tf.keras.regularizers.l2(0.01)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu',kernel_initializer = tf.keras.initializers.HeNormal(),
                     kernel_regularizer=tf.keras.regularizers.l2(0.01)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(64, activation='relu',kernel_initializer = tf.keras.initializers.HeNormal(),
                    kernel_regularizer=tf.keras.regularizers.l2(0.01)))
    model.add(Dense(10, activation='softmax'))
    # compile model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    # fit model with generator
    model.fit_generator(train_iterator, steps_per_epoch=len(train_iterator), epochs=5)
    # evaluate model
    _, acc = model.evaluate_generator(test_iterator, steps=len(test_iterator), verbose=0)
    print('Test Accuracy: %.3f' % (acc * 100))

In [124]:
L2Regularization()

Train min=0.000, max=255.000
Test min=0.000, max=255.000
Batches train=938, test=157
Batch shape=(64, 28, 28, 1), min=0.000, max=1.000
Epoch 1/5
938/938 [==============================] - 17s 17ms/step - loss: 2.1105 - accuracy: 0.7528
Epoch 2/5
938/938 [==============================] - 17s 18ms/step - loss: 0.7548 - accuracy: 0.8245
Epoch 3/5
938/938 [==============================] - 17s 18ms/step - loss: 0.6756 - accuracy: 0.8281
Epoch 4/5
938/938 [==============================] - 16s 17ms/step - loss: 0.6412 - accuracy: 0.8336
Epoch 5/5
938/938 [==============================] - 16s 18ms/step - loss: 0.6194 - accuracy: 0.8369
Test Accuracy: 83.200
